**注意事項**

このノートブックは、GPU:「T4」に対応させたものです。
「L4」版のノートブックとはモデル等が異なるため、生成される内容が異なることが考えられます。

生成される内容と、ノートブックに記載されている説明が一致しない場合があることをご了承ください。

生成内容とノートブックの説明をよく見比べ、適宜読み替えながら演習を進めてみてください。

---

# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「Inference Time Scaling（推論時スケーリング）」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 2024年8月発表の論文「Scaling LLM Test-Time Compute Optimally can be More Effective than Scaling Model Parameters」で提唱された概念
- OpenAIのGPT-o1（2024年9月リリース）で実用化され、注目を集めた比較的新しいアプローチ
- 2024年度LLM講座の第4回講義でも取り上げられた重要テーマ

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [53]:
# !pip install --upgrade transformers
# !pip install google-colab-selenium
# !pip install bitsandbytes

In [54]:
# 演習用のコンテンツを取得
# !git clone https://github.com/matsuolab/lecture-ai-engineering.git

In [55]:
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())

!huggingface-cli login --token $$HUGGINGFACE_TOKEN

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: huggingface-cli <command> [<args>] login [-h] [--token TOKEN]
                                                [--add-to-git-credential]
huggingface-cli <command> [<args>] login: error: argument --token: expected one argument


In [56]:
# # HuggingFace Login
# from huggingface_hub import notebook_login

# notebook_login()

In [57]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [58]:
import random
random.seed(0)

In [59]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.37s/it]


In [60]:
answers =[]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [61]:
messages = [
    {"role": "user", "content": "LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [62]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、Large Language Model (LLM) の推論時間（Inference Time）を効率的に調整する方法です。 

**従来の推論時間:**

* LLMの複雑な計算処理により、推論時間が非常に長くなる傾向があります。
* リアルタイムでの応答が難しい、複雑な処理が必要な場合に大きな課題となります。

**Inference Time Scalingの目的:**

* **推論時間を短縮:**  より迅速な応答を実現し、ユーザーの満足度を高めます。
* **コスト削減:**  推論時間を短縮することで、計算資源の消費量を削減できます。
* **スケーラビリティ向上:**  大量のユーザーからのリクエストに対応できるよう、システムのスケーラビリティを高めます。

**Inference Time Scalingを実現するための方法:**

* **モデルの最適化:**  モデルの構造やパラメータを調整することで、推論時間を短縮できます。
* **データの最適化:**  推論に必要となるデータのサイズや構造を最適化することで、処理時間を削減できます。
* **ハードウェアの最適化:**  高速なCPUやGPU、メモリ


In [63]:
answers.append(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「Inference Time Scaling」について誤った知識を提示しました：
* モデルは従来の「推論時間の短縮」という文脈でInference Time Scalingを解釈しており、これはLLM分野における最新の「Inference Time Scaling」概念（推論時計算資源の最適配分）とは異なる説明になります。

---

# 2. 文字起こしデータの活用
## 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: LLM講座第4講における講師の発言内容
* **目的**: モデルに「Inference Time Scaling」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 音声認識モデル(speech2text)で書き起こした生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [64]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

In [65]:
with open("./data/LLM2024_day4_raw.txt", "r") as f:
# with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4_raw.txt", "r") as f:
  raw_writedown = f.read()

In [66]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[250])

ドキュメントサイズ:  306
ドキュメントの例: 
 このDecodingにもいろんな方法があってグリーンDecodingだと単純に一番いいやつを選んでいく、一番確率が高いやつ選んでいくので、すごい単純ですけど、こういうトップKeyを取るとかトップ系を取るとかして、最後に一番いいやつを選ぶみたいなことをすると、これも結局計算をたくさんしてることになるわけですね


In [67]:
# Retrievalの実行
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[61.58913803100586, 65.9614028930664, 59.17729187011719, 53.388343811035156, 51.554901123046875, 58.392417907714844, 60.75017547607422, 54.14458465576172, 61.971519470214844, 62.857818603515625, 58.387691497802734, 59.10322189331055, 55.33921813964844, 57.35508728027344, 59.75800704956055, 58.144813537597656, 62.519622802734375, 57.498802185058594, 56.666446685791016, 59.71482849121094, 59.4350471496582, 64.46308135986328, 62.21357727050781, 60.33452606201172, 53.73405075073242, 64.26007843017578, 57.02006912231445, 62.448455810546875, 56.579444885253906, 56.528099060058594, 56.84428405761719, 54.060768127441406, 51.05217742919922, 51.92462158203125, 56.22425079345703, 56.69752502441406, 57.82189178466797, 57.06683349609375, 59.669166564941406, 59.15591049194336, 57.53609085083008, 52.950382232666016, 55.696495056152344, 54.76655960083008, 58.1320686340332, 61.39439392089844, 58.75428771972656, 61.9227409362793, 56.53096389770508, 58.76028060913086, 59.051002502441406, 53.050468444824

In [68]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 67.52084350585938)
Trasnformerの場合はスケール則がごめんなパラメータ数が横軸になってますけどこういうふうになると、LLMの場合の一掃にソヨンそうなのでそれぞれ計測と書くとこんなふうになりますよということで、Trasnformer以外のスケール則っていうのもあの研修をされて、深さについても検証してまして、これも他のモデルが何だったかちょっと忘れちゃったけどリスキーだったような気がしますけどそう変えたときにどういうふうな変化するかっていうのをこういった形でプロットするようなGENIACすることができます 


取得したドキュメント2: (Score: 66.03305053710938)
1月に論文としてまして、スケーリングLLMthisTimeコンピュート口真理ちゃん日は増えてるっていうことで、良いらしいというふうに言われてます 


取得したドキュメント3: (Score: 65.9614028930664)
あのスケールするっていうところではタイトルの通りなんですけど、ちょっとこれスケーリングPretrainingってなってるんですけれども、ちょっと最近はですね、このPretrainingだけではなくて、推論をスケールさせるというような話も出てきてましてせっかくなのでその最近の話題ということですレンジのスケーリングことでちょっとタイトル詐欺が入ってるんですけどPretrainingだけじゃない、スケーリングを扱うということでちょっと若干あのタイトル詐欺なんですけども、あの最近の話題ということで水土日のスケジュールについても話していきたいなと思っています 


取得したドキュメント4: (Score: 65.18775177001953)
気にしながらっていうのの実例を出した方がわかりやすいと思うので、実際にこれ開発者じゃないので、あの結果を見て推論してるだけなんで嘘ついてるかもしれないですけど例えばLlama3の論文を持ってくると8Billon70Billon405Billonって層の数モデルDimension埋め込みの数次元ですね、フィードフォワードの次元、アテンションの数っていうのを、こういうふうにしたよっていうふうに言われてます 


取得したドキュメント5: (Score: 65.13

In [69]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [70]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**は、**LLM（Large Language Model）の推論時間（Inference Time）をスケールする手法**です。 

**簡単に説明すると:**

* **スケール:**  LLMの処理能力を拡張し、より多くのデータや複雑なタスクを処理できるようになることを意味します。
* **Inference Time Scaling:**  スケールを適用することで、推論時間を効率的に短縮し、より高速で応答できるようになります。

**具体的な内容:**

* **Pretraining:**  LLMの学習段階で、大量のデータからモデルを学習させる手法。
* **スケーリングPretraining:**  Pretraining段階だけでなく、推論段階においてもスケールを適用することで、より高速な推論が可能になります。
* **スケーリング:**  推論段階におけるスケール化手法は、モデルの層数や次元数、フィードフォワードの次元、アテンションの数などを調整することで、推論時間を効率的に短縮します。

**最近の注目点:**

* **スケーリングPretraining:**  Pretraining段階だけでなく、推論段階においてもスケールを適用することで、より


In [71]:
answers.append(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (初期RAG実装)

講義内容のドキュメントを追加したにもかかわらず、モデルの回答には依然として以下の問題が見られます：
* 「高速に推論する」など、従来の一般的な推論最適化と「Inference Time Scaling」を混同した誤った解釈が継続
* 講義内容を参照しているものの、概念の本質を正確に捉えられていない

### 問題分析
以下の要因が考えられます：
1. **ドキュメント品質の問題**: 音声認識による文字起こしの精度不足
2. **検索精度の課題**: 単純な文単位の分割では文脈が失われ、関連性の高いドキュメント片を適切に取得できていない可能性

### 書き起こしテキストの品質改善

日本語の音声認識（speech2text）モデルは精度に課題があることが知られています。以下に「LLMにおけるInference Time Scalingとは？」に関連する講義内容の書き起こしテキストを比較します：

### 講義中の該当発言 (LLM講座Day4後半から抜粋)


<修正前>
---

講義に戻ります。ちょっと練習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドって話で**生のGENIACLM**の話をして終わろうと思いですねちょっとモチベーションから話すと、ちょっと頭で考えてみてほしいとか見れば一瞬で思うとんですけどバナナの色は何ですかって言われたときと、今日の講義聞いた上で、**ゲームソフトの問題は何だと思いますか**って聞かれたとき、多分あの考えることが違うと思うんですね。**羽の色なんですか**っていうと一瞬黄色ですねもしかしたら緑かもしれないけどぐらいですかね物によるかなみたいなおもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**スケール足の問題なんだと思いますか**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだからどうだろうこの辺が問題かなみたいな考えとやっぱ思考としては違うってことは何となく思うかなと思います。なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**スケールって言ってる7Gのスケール**って言ってるのはこういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、あの考えなきゃいけない問題に対しては、考える時間を、に計算式を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。二つで、ちょっと順番が前後しますけどこれの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**大湾**と呼ばれる論文が論文じゃないか、モデルが**オペル**から出ましたプレビューとして出てますけどこの法案で注目されていますこれあの**論文にROMってかブログ**にあるとイエスって右側が訓練時の計算資源をスケールさせたときに、初めて何かロジックのベンチマークがあるんですけどこれをがどうなったかで何となくスケールしてると右側がテストTimeコンピュートっていうふうに書いてると思うんすけど、**水温時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだんコース計算式を増やしていったときに、性能がどう変わるかっていうのでこれもスケールしていってるということがわかると思います。こういった形で、要は考える時間をどうやら推論時に使うと計算資源を推論使うのはいいことがありそうだということがわかります。


<修正後>
---


講義に戻ります。ちょっと演習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドってことで**「推論時のスケーリング」**についての話をして終わろうと思います。モチベーションから話すと、ちょっと頭で考えてみてもらえれば一瞬でわかると思うとんですけど、「バナナの色は何ですかって言われたとき」と、今日の講義聞いた上で、**「スケール則の問題は何だと思いますか」**って聞かれたとき、多分あの考えることが違うと思うんですね。
**「バナナの色なんですか」**っていうと黄色ですね。もしかしたら緑かもしれないけど、物によるかなみたいな、おもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**「スケール則の問題なんだと思いますか」**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだから「どうだろう」「この辺が問題かな」みたいな考えとはやっぱ思考としては違うってことは何となく思うかなと思います。
なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**推論時のスケールって言ってるのは**こういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、深く考えなきゃいけない問題に対しては、考える時間に計算資源を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。
これの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**o1**と呼ばれるモデルが**OpenAI**から出ました。プレビューとして出てますけどこのo1で注目されています。これあのo1の**論文ってかブログ**にある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。右側がtest-time computeっていうふうに書いてると思うんすけど、**推論時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います。
こういった形で、要は考える時間をどうやら推論時に使うと、つまり計算資源を推論時に使うのはいいことがありそうだということがわかります。






---
### 文字起こしの誤り

上記の比較からわかるように、音声認識による書き起こしには重大な誤りが多数含まれています：
* 「スケール則の問題」→「ゲームソフトの問題」
* 「o1」→「大湾」
といった明らかに文脈に合わない単語変換が発生しています。

`LLM2024_day4_raw.txt`の中には、このような誤変換が多数見られます。これらの誤りはRAG性能に直接影響し、モデルの回答精度を低下させる要因となります。

したがって、**ドキュメント品質の改善**を行い、RAG性能の向上を図ります。

## 講義内容をソースとして活用：改善版RAG実装

* **ドキュメント処理**: 
  - speech2textによる書き起こしテキストを人手で丁寧に修正
  - 専門用語（Inference Time Scaling、GPT-o1など）の正確な表記を確保
  - 文脈の流れを維持しつつ、文法的に正確な日本語に修正

* **検索手法**: 
  - 引き続き「。」（句点）で区切られた文単位でテキストを分割
  - 文単位の検索により、モデルの入力トークン制限内で関連情報を最大化

この改善により、モデルが正確な情報に基づいて「Inference Time Scaling」の概念を理解し、適切な回答を生成することが期待されます。

In [72]:
with open("./data/LLM2024_day4.txt", "r") as f:
# with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4.txt", "r") as f:
  raw_writedown = f.read()

In [73]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[310])

ドキュメントサイズ:  350
ドキュメントの例: 
 それからBest of Nとはちょっと違う方法として、N個を生成した後に、それらを集約するという意味では、Day2でやったSelf-Consistencyをこの枠組みの一つとして説明されます


In [74]:
# Retrievalの実行
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[61.822235107421875, 66.46027374267578, 60.23433303833008, 51.8294563293457, 54.90752410888672, 51.55607604980469, 58.39425277709961, 60.800106048583984, 55.06976318359375, 59.28041458129883, 58.47063064575195, 62.85796356201172, 58.220882415771484, 59.10316848754883, 55.34014892578125, 57.35477066040039, 59.756874084472656, 51.492427825927734, 60.6212272644043, 62.5194091796875, 57.498939514160156, 58.55681228637695, 59.716163635253906, 59.434837341308594, 63.48379898071289, 61.778526306152344, 59.61750030517578, 62.45526123046875, 53.734352111816406, 61.902618408203125, 55.2577018737793, 56.98680877685547, 62.4495735168457, 56.231964111328125, 56.5261116027832, 56.395809173583984, 58.69124984741211, 51.74648666381836, 56.2239875793457, 56.69636154174805, 58.36695861816406, 55.44096374511719, 59.27631759643555, 59.533382415771484, 57.69160842895508, 54.764198303222656, 58.02476501464844, 57.39853286743164, 56.447208404541016, 52.43053436279297, 61.59440231323242, 62.69723129272461, 5

In [75]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 67.37751770019531)
最後に補足して僕のパート終わろうと思いますけど、同じ計算資源のときにパラメータ増やすのよりも推論資源を増やすのが有効なのかっていうのが問いとしてあると思いますけど、o1の場合だと、訓練時のスケールは同じままって推論時のスケールを増やしたら、より賢くなりましたって話でしたけど、どっちにするのがいいのかっていう意味で言うと、GoogleDeepMindが8月に論文としてまして、Scaling LLM Test-Time Comupte Optimally can be more Effective than Scaling More Paremetersっていうことで、良いらしいというふうに言われてます 


取得したドキュメント2: (Score: 66.5853042602539)
右側がtest-time computeっていうふうに書いてると思うんすけど、推論時に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います 


取得したドキュメント3: (Score: 66.46027374267578)
あのスケールするっていうところではタイトルの通りなんですけど、ちょっとこれスケーリングPretraining回ってなってるんですけれども、ちょっと最近はですね、このPretrainingだけではなくて、推論をスケールさせるというような話も出てきてましてせっかくなのでその最近の話題ということです推論時のスケーリングことで、ちょっとタイトル詐欺が入ってるんですけどPretrainingだけじゃない、スケーリングも扱うということで、ちょっと若干あのタイトル詐欺なんですけども、あの最近の話題ということで推論時のスケジュールについても話していきたいなと思っています 


取得したドキュメント4: (Score: 66.20555877685547)
Trasnformerの場合はスケール則が、パラメータ数が横軸になってますけどこういうふうになると、LSTMの場合には1層2層4層みたいにそれぞれスケール則を解くとこんなふうになりますよ

In [76]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [77]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時に計算資源を増やす**ことを指します。 

**具体的には:**

* **Pretraining**: モデルを訓練する際に使用する計算資源
* **Inference**: モデルを実際に利用する際に使用する計算資源

**Inference Time Scalingのメリット:**

* **スケールする**:  モデルの規模を大きくするのではなく、推論時の計算資源を増やすことで、性能を向上させることが可能になります。
* **パラメータ数の増加よりも効果的**: GoogleDeepMindの研究では、推論時の計算資源を増やす方が、より効果的であることが示唆されています。


**Inference Time Scalingの具体的な方法:**

* **Pretraining**: モデルの学習時に使用する計算資源
* **Inference**: モデルの利用時に使用する計算資源


 





In [78]:
answers.append(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (修正テキストによるRAG)

書き起こしテキストの品質改善により、モデルの回答に部分的な向上が見られました：

### 改善点
* 「推論時（Inference）に計算資源をスケーリングすることで、モデルがより賢くなり、性能が向上すること」という概念を正確に捉えるようになった

### 問題点
* 「Inference Time Scalingは、TransformerやLSTMなどのモデルにおいて、パラメータ数を増やすのではなく、推論時計算資源をスケーリングすることで、性能が向上すること...」という記述は講義内容と矛盾している

### 問題分析

モデルが誤った回答を生成する主要因として、**文脈の欠如**が考えられます：
* 「。」で区切られた短い文単位での検索では、各文の発言背景や関連性が失われる
* 単独の文から情報を抽出するため、講師の全体的な主張や議論の流れを把握できない
* 結果として、正しい個別の文でも、その解釈に必要な背景情報が欠如し、誤った文脈で理解される


---

# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [79]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [80]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時の計算資源を増やすことで、モデルの性能を向上させる**ことを指します。 

具体的には、**モデルの学習段階（Pretraining）で計算資源を増やす**だけでなく、**推論段階（Inference）で計算資源を増やす**という方法も注目されています。


**Inference Time Scalingの目的**

* **計算資源の効率性向上:**  推論時に必要な計算資源を効率的に使用することで、コスト削減と実行時間の短縮を実現します。
* **モデル性能の向上:**  推論時の計算資源を増やすことで、モデルの精度や処理能力を向上させることが期待されます。



**Inference Time Scalingの現状**

* **Pretraining段階でのスケール:**  従来のモデルでは、Pretraining段階で計算資源を増やすことが主流でした。
* **推論段階でのスケール:**  近年、推論段階でのスケールも注目され、より効率的なモデル開発が進んでいます。
* **スケール則の研究:**  スケール則を研究することで、最適なスケール方法を特定し、より効率的なモデル開発を実現します。



**Inference Time


In [81]:
answers.append(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られました：

### 改善点
* 「推論時の計算をスケールさせる」という概念を据えて回答
* Inference Time Scalingの基本原理についての理解が向上

### 残存する問題点
* 質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇


---

# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『LLMにおけるInference Time Scalingとは？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、Inference Time Scalingに関する正確で一貫性のある回答生成が期待されます。

In [82]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = []
for ref in ["。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:
  messages = [
      {"role": "user", "content": f"与えられた[参考資料]が[質問]に直接関連しているかを、'yes''no'で答えること。[参考資料]\n{ref}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      # max_new_tokens=128,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  response = tokenizer.decode(response, skip_special_tokens=True)
  print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
  print("\n関連しているかどうか: ", response)

  if "yes" in response.lower():
    references.append(ref)



対象となるドキュメント:
 これでほぼちょうどですけど、最後に少しあの、前半では全体の訓練時のスケーリングをする話を基本的にしましたけど、最近ではこの推論時の計算量っていうのも注目するような研究が増えてきています。
代表的なGPT-o1とかですごく注目されてるかなと思いますし、今までやった方法、学んだ方法も結構出てきたと思いますけど、Promptingを工夫するとか、Decodingを工夫するとかいうので、それにも発展的な方法がいろいろ出てきていますし、Meta Generationっていうような枠組みで、DecodingだけじゃなくてそのDecodeした結果を最後どう使うかみたいな含めて、Meta Generationというふうに呼んでますけど、Paralell SearchとかStep Level SearchとかRefinementと言われるような枠組みの研究も出てきていますというような話をしました。
最後に補足して僕のパート終わろうと思いますけど、同じ計算資源のときにパラメータ増やすのよりも推論資源を増やすのが有効なのかっていうのが問いとしてあると思いますけど、o1の場合だと、訓練時のスケールは同じままって推論時のスケールを増やしたら、より賢くなりましたって話でしたけど、どっちにするのがいいのかっていう意味で言うと、GoogleDeepMindが8月に論文としてまして、Scaling LLM Test-Time Comupte Optimally can be more Effective than Scaling More Paremetersっていうことで、良いらしいというふうに言われてます。
厳密に言うとこれなんかタスクによって違うということなので、良いとまで言っていいのかちょっと若干誇大広告な気が個人的にはしてますけど、そういったことを検証するような研究も出てきていますので興味ある人は見てもらえばと思います

関連しているかどうか:  yes 





対象となるドキュメント:
 プレビューとして出てますけどこのo1で注目されています。
これあのo1の論文ってかブログにある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。
右

In [83]:
print(len(references))

4


上記より、上位4件のみが関連しているとわかったので、これらだけをモデルに渡すこととする。（生成内容が確立的なので、4件でない可能性もあります）

In [84]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
messages = [
    {"role": "user", "content": f"与えられる資料を参考にして回答すること。[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [85]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## LLMにおけるInference Time Scalingとは？

Inference Time Scalingとは、**推論時に計算資源を増やす**ことを指します。 

**具体的には:**

* **Pretraining**段階でモデルを学習させる際に、計算資源を増やす
* **Inference**段階で、モデルを実際に利用する際に、計算資源を増やす

**目的:**

* **性能向上:** 推論速度を高速化し、より迅速な回答を実現
* **スケール性の拡張:**  より複雑なタスクに対応できるようになる


**最近の研究:**

* **Pretrainingのみでは限界:**  モデルの規模が大きくなると、Inference Time Scalingが重要になってくる
* **Inference Time Scalingの効果:**  Pretraining段階でのスケールと比較して、Inference Time Scalingの方が効果的であることが示唆されている


 





In [86]:
answers.append(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

### 達成された成果
* Inference Time Scalingに関する正確な情報を含んだ回答の生成
* 無関係な情報やノイズの排除
* 講義内容を反映した説明の実現 🎉

この結果から、RAGパイプラインにおける情報の質と関連性の重要性であり、検索で取得した情報を単に増やすだけでなく、その情報の関連性を精査する方法を学ぶことができました。

---


# 5. さらなる改善案: 意味的チャンク化

文単位での分割と前後文脈の追加という現在のアプローチをさらに発展させる手法として、**意味的なチャンク化**が考えられます：

* **意味的チャンク（段落）単位での分割**:
  - 単純な文の区切りではなく、意味的なまとまり（トピック、議論、例示など）に基づいてテキストを分割
  - 人間の主観に基づく意味的な段落分けを活用
  - 各チャンクが「一つの完結した考え」を表現するようにする

* **期待される効果**:
  - より自然な文脈理解が可能に（人間の思考や会話の流れに近い）
  - トピックの開始から結論までの流れを維持できる
  - 概念間の関係性や比較が同一チャンク内に含まれ、より深い理解につなげる

* **検証方法**:
  - 人間が主観的に意味でグループ化したチャンクセットを用意
  - 同じRerank手法を適用し、文単位チャンクとの性能差を比較
  - 回答の正確性、一貫性、網羅性を評価指標として使用

この意味的チャンク化手法は、特に講義のような構造化された発話においては、より自然で効果的な情報検索と理解を可能にすると予想されます。

**注意事項**

**ここから先のセルを実行した場合、GPUメモリ不足になる可能性が高いです。**


このノートブックでは、GPUはT4を使用しています。
Colab Pro等を契約し、L4などのよりGPUメモリの大きいものを使用するか、モデルやその設定等を変更するなどの工夫が必要になります。

In [87]:
# 本来は段落をそのままdocumentsに入れずに一定のサイズに分割した方が良いでしょうが、簡単のために段落をそのまま入れてしまいます。
documents = [text.replace("\n", " ").strip() for text in raw_writedown.split("\n\n")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[30])

ドキュメントサイズ:  45
ドキュメントの例: 
 具体的な求め方についても話します。 さっきからチラチラ言ってた通りなんすけど基本的にこれどう図るかっていうと、基本的にはいくつかの条件で実験してフィッティングするって言ってんのは、すごい単純に言ってしまうとそうなります。左側GPT4の論文から取ってきた図で説明したもんですけど、グレーのやつを例えば実験してみて、これぐらいのロスになるんだなっていうので、フィッティングするとこういうカーブになります。 ちなみにこれ、なんでこれ直線にならないんだっていうのをすぐ説明しなかったですがこれ縦軸が実は普通のロスと違ってBits-per-wordっていうのになってて、多分2乗スケールのロスになってるからだと思います。 右側も同じですね。この各点について何かいろんな設定で実験してやって、それを結果を見るということをしてますけどよくよく考えるとスケールさせるときにモデルサイズどうすればいいんでしたっけとか、何をどういじるとモデルサイズが大きくなるんでしたっけ、どういうふうに言えばいいんでしたっけとかですね。 あのモデルサイズ変えたときにハイパーパラメータってどうすんでしたっけそういった細かい問題が出てくる。最初の方ですけどモデルサイズどう変化させるかっていうので、前回やった、こういう図があると思いますけどモデルサイズ変えようと思ったら別にパラメータ、層の数を増やしても、いいわけですし、この埋め込みの次元各tokenの次元を増やしてもいいわけですし、各随所に出てくるこのフィードフォワードネットワークっていうのの中間層の次元を上げてもいいですしヘッドを増やしてもそういうのあのパラメータ自体は上がるということで、これどれをどのぐらいやるんですかっていうのが細かく考えると重要になってきます。 この辺は元の論文でも一応議論されてまして、これ三つほど出してるんすけど例えば真ん中のがアスペクト比っていう、モデルのエンベディングのサイズですね。dモデルっていうものを層数で割ったもの、アスペクト比という縦横比みたいなもので幅と深さの比率をアスペクト比っていうふうにこの論文では呼んでいますけど。こういったものを変えて実験してみたっていうのが最初の最初じゃないOpenAIのScaling Lawで話されていました。基本的にはこの辺見るとなんかあんまり性

In [88]:
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[64.80940246582031, 61.82463073730469, 62.279415130615234, 62.75594711303711, 63.07190704345703, 61.71656799316406, 59.56270217895508, 59.89581298828125, 61.3881721496582, 62.31979751586914, 61.01363754272461, 65.68667602539062, 65.840576171875, 60.71131134033203, 62.971527099609375, 62.90633010864258, 61.8266716003418, 63.1424560546875, 60.52707290649414, 59.37120819091797, 59.915531158447266, 63.584476470947266, 59.79716873168945, 58.89724349975586, 62.60258102416992, 63.643890380859375, 61.1680793762207, 61.81727600097656, 60.26072692871094, 61.51399612426758, 64.97259521484375, 63.2974967956543, 63.33731460571289, 63.41263198852539, 65.70318603515625, 64.9515380859375, 58.451175689697266, 60.1381721496582, 60.07516860961914, 58.61343765258789, 57.92119598388672, 59.45044708251953, 58.8979377746582, 63.05853271484375, 68.84087371826172]]


In [89]:
# 簡単のためにtop2でやります。結果を見てもらえれば問題なく関連する項目のみ取得できているのが分かるかと思います。
topk = 2
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 68.84087371826172)
最後に補足して僕のパート終わろうと思いますけど、同じ計算資源のときにパラメータ増やすのよりも推論資源を増やすのが有効なのかっていうのが問いとしてあると思いますけど、o1の場合だと、訓練時のスケールは同じままって推論時のスケールを増やしたら、より賢くなりましたって話でしたけど、どっちにするのがいいのかっていう意味で言うと、GoogleDeepMindが8月に論文としてまして、Scaling LLM Test-Time Comupte Optimally can be more Effective than Scaling More Paremetersっていうことで、良いらしいというふうに言われてます。厳密に言うとこれなんかタスクによって違うということなので、良いとまで言っていいのかちょっと若干誇大広告な気が個人的にはしてますけど、そういったことを検証するような研究も出てきていますので興味ある人は見てもらえばと思います。 


取得したドキュメント2: (Score: 65.840576171875)
それから元のOpenAIの論文に戻りますと、今言ったようなLSTMの比較みたいなLSTMにおけるスケール則みたいなことも、この論文でも検証されていまして、左側がモデル構造が違うんですね。 Trasnformerの場合はスケール則が、パラメータ数が横軸になってますけどこういうふうになると、LSTMの場合には1層2層4層みたいにそれぞれスケール則を解くとこんなふうになりますよということで、Trasnformer以外のスケール則っていうのもあの検証をされている。深さについても検証してまして、これも元のモデルが何だったかちょっと忘れちゃったけど、確かLSTMだったような気がしますけど、層を変えたときにどういうふうな変化するかっていうのをこういった形でプロットするようなことがされてます。 ポイントはTrasnformer以外でも別にあのスケールするっていうのは成立概念だということです。なんでこんなTrasnformerだけ注目されてるのかってのは後で話します。 




In [90]:
reference = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])

messages = [
    {"role": "user", "content": f"[参考資料]\n{reference}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [91]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

**LLMにおけるInference Time Scalingとは、推論時の計算資源を増やすことで、モデルの


In [92]:
answers.append(tokenizer.decode(response, skip_special_tokens=True))

In [93]:
print(len(answers))
for i, answer in enumerate(answers):
    print(f"Answer {i + 1}: {answer[:150]}")

6
Answer 1: ## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、Large Language Model (LLM) の推論時間（Inference Time）を効率的に調整する方法です。 

**従来の推論時間:**

* LLM
Answer 2: ## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**は、**LLM（Large Language Model）の推論時間（Inference Time）をスケールする手法**です。 

**簡単に説明すると:**

* **ス
Answer 3: ## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時に計算資源を増やす**ことを指します。 

**具体的には:**

* **Pretraining**: モデルを訓練する際に使用する計算資源
* **Inf
Answer 4: ## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時の計算資源を増やすことで、モデルの性能を向上させる**ことを指します。 

具体的には、**モデルの学習段階（Pretraining）で計算資源を増やす**だ
Answer 5: ## LLMにおけるInference Time Scalingとは？

Inference Time Scalingとは、**推論時に計算資源を増やす**ことを指します。 

**具体的には:**

* **Pretraining**段階でモデルを学習させる際に、計算資源を増やす
* **Infe
Answer 6: **LLMにおけるInference Time Scalingとは、推論時の計算資源を増やすことで、モデルの


In [106]:
import itertools
from collections import defaultdict

class LLMJudge:
    def __init__(self, model, tokenizer):
        self.tokenizer = tokenizer
        self.model = model
        self.device = model.device
        self.terminators = self.tokenizer.eos_token_id

    def build_prompt(self, query, answer_a, answer_b):
        
        return [
            {"role": "user", "content": f"""
あなたは中立で正確なAI審査員です。以下の質問と2つの回答（AとB）を比較し、どちらがより優れているかを評価してください。評価基準は、正確さ、具体性、わかりやすさです。
質問: {query}

回答 A: {answer_a}
回答 B: {answer_b}

どちらが優れていますか？「A」または「B」とだけ出力してください。
"""}
        ]
        
    def evaluate_pair(self, query, answer_a, answer_b):
        prompt = self.build_prompt(query, answer_a, answer_b)
        # print(f"Prompt: {prompt}")
  
        input_ids = tokenizer.apply_chat_template(
                        prompt,
                        add_generation_prompt=True,
                        return_tensors="pt"
                    ).to(model.device)
        
        # inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        # print(f"Inputs: {inputs}")

        with torch.no_grad():
            outputs = self.model.generate(
                input_ids,
                eos_token_id=self.terminators,
                max_new_tokens=32,
                do_sample=False
            )
        # print(f"Outputs: {outputs}")
        generated_text = self.tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        # print(f"Generated text: {generated_text}")
        # 結果から判定文字列を抽出
        if "A" in generated_text or "B" in generated_text:
            if "同等" in generated_text:
                return "同等"
            elif "A" in generated_text.split()[0]:
                return "A"
            elif "B" in generated_text.split()[0]:
                return "B"
        return "同等"  # fallback

class AnswerRanker:
    def __init__(self, query, answers, judge: LLMJudge):
        self.query = query
        self.answers = answers
        self.judge = judge
        self.win_count = defaultdict(int)

    def run_evaluation(self):
        for i, j in itertools.combinations(range(len(self.answers)), 2):
            answer_a, answer_b = self.answers[i], self.answers[j]
            result = self.judge.evaluate_pair(self.query, answer_a, answer_b)
            print(f"Comparing #{i} vs #{j} -> {result}")
            if result == "A":
                self.win_count[i] += 1
            elif result == "B":
                self.win_count[j] += 1
            # 同等 → 何もしない

    def get_ranking(self):
        ranked_indices = sorted(range(len(self.answers)), key=lambda i: -self.win_count[i])
        return [(idx, self.win_count[idx], self.answers[idx]) for idx in ranked_indices]

    def print_ranking(self):
        print("\n=== RANKING ===")
        for rank, (idx, wins, ans) in enumerate(self.get_ranking(), start=1):
            print(f"{rank}. Answer #{idx} (Wins: {wins})")
            # print(f"    {ans}")

In [107]:
query="[質問] LLMにおけるInference Time Scalingとは？"

judge = LLMJudge(model, tokenizer) 
ranker = AnswerRanker(query, answers, judge)

ranker.run_evaluation()
ranker.print_ranking()

Comparing #0 vs #1 -> A
Comparing #0 vs #2 -> A
Comparing #0 vs #3 -> A
Comparing #0 vs #4 -> A
Comparing #0 vs #5 -> A
Comparing #1 vs #2 -> A
Comparing #1 vs #3 -> A
Comparing #1 vs #4 -> A
Comparing #1 vs #5 -> A
Comparing #2 vs #3 -> A
Comparing #2 vs #4 -> B
Comparing #2 vs #5 -> A
Comparing #3 vs #4 -> A
Comparing #3 vs #5 -> A
Comparing #4 vs #5 -> A

=== RANKING ===
1. Answer #0 (Wins: 5)
2. Answer #1 (Wins: 4)
3. Answer #2 (Wins: 2)
4. Answer #3 (Wins: 2)
5. Answer #4 (Wins: 2)
6. Answer #5 (Wins: 0)
